# Assessing Public opinion of a company through Financial tweets

* This research intends to replicate the methods of the study titled: "Assessing Public Opinions Through Web 2.0: A Case Study on Wal-mart"

* Although, for this context, I will be engaging a different dataset retrieved from Kaggle: https://www.kaggle.com/datasets/davidwallach/financial-tweets/data?select=stockerbot-export.csv. 

* The dataset for the original study is not available at the highlighted url. The forum was inactive.

In [28]:
#import libraries
import pandas as pd
import numpy as np

In [29]:
#names=["id","text","timestamp","source","symbols","company_names","url","verified"]
#used on_bad_lines='skip' to escape text with special characted causing an issue with file to be read
#downside of this is losing data

raw_data = pd.read_csv('./Downloads/archive-3/stockerbot-export.csv',on_bad_lines='skip')

In [30]:
#find number of rows
raw_data.shape

(28264, 8)

*** Data has 28624 rows and 8 attributes ("id","text","timestamp","source","symbols","company_names","url","verified")

In [31]:
#view data
raw_data.head()

,id,text,timestamp,source,symbols,company_names,url,verified
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,True
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,True
2,1019711413798035500,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,AIG,American,https://buff.ly/2L3kmc4,True
3,1019716662587740200,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,True
4,1019718460287389700,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,True


### Exploratory Analysis

In [32]:
#get column names
raw_data.columns

Index(['id', 'text', 'timestamp', 'source', 'symbols', 'company_names', 'url',
       'verified'],
      dtype='object')

In [33]:
# How many companies are present in this dataset?
raw_data['company_names'].value_counts()

Twenty-First Century Fox        131
Alphabet Inc.                   116
Discovery                       102
Netflix                         101
Honeywell International Inc.    100
                               ... 
Facebook*Alphabet*Alphabet        1
Intel*U.S.                        1
Harris                            1
JD                                1
The Goldman Sachs                 1
Name: company_names, Length: 461, dtype: int64

In [34]:
raw_data.groupby('company_names').count()['id']

company_names
 name              1
3M Company        60
ANSYS             37
AT&T Inc.         44
AbbVie Inc.       95
                  ..
Zoetis Inc.       75
Zynga Inc.        46
eBay               1
eBay Inc.         79
salesforce.com    54
Name: id, Length: 461, dtype: int64

In [38]:
pd.DataFrame(raw_data['symbols'].value_counts())

,symbols
NFLX,101
AMAT,100
GPS,100
MOMO,100
HON,100
...,...
INTC-USB,1
AMZN-GPS,1
FB-GOOGL-GOOG,1
M,1


In [36]:
#store text frequency result in a dataframe
company_txt_frq = pd.DataFrame(raw_data['symbols'].value_counts())

In [37]:
#view
company_txt_frq.head(20)

,symbols
NFLX,101
AMAT,100
GPS,100
MOMO,100
HON,100
MTB,100
ES,100
MAS,99
ESS,99
GRPN,99


* Nextflix has the highest frequency of text in this dataset
* We subset the data to focus only on text on Netflix to streamline the analysis

In [43]:
#retrieve data on Netflix only for further analysis
Netflix_text_data = raw_data[raw_data["company_names"]=="Netflix"]

In [44]:
#view first five rows
Netflix_text_data.head()

,id,text,timestamp,source,symbols,company_names,url,verified
8,1019720723441635300,Senate wants emergency alerts to go out throug...,Wed Jul 18 23:09:00 +0000 2018,TechCrunch,NFLX,Netflix,https://tcrn.ch/2L8DsgT,True
26216,1019690098236055600,Netflix $NFLX just released quarterly 10-Q. Q...,Wed Jul 18 21:07:19 +0000 2018,Feedmeistercom,NFLX,Netflix,NaN,False
26223,1019690300187533300,RT @blackbullforex: Excluding FAANG Stocks The...,Wed Jul 18 21:08:07 +0000 2018,dexoidan,NFLX,Netflix,NaN,False
26236,1019690538398707700,RT @RampCapitalLLC: Here is where $FANG was tr...,Wed Jul 18 21:09:03 +0000 2018,hs_traders,NFLX,Netflix,NaN,False
26239,1019690604559851500,RT @SevenPointsCap: Trader Takeaways 7.18.18 b...,Wed Jul 18 21:09:19 +0000 2018,clovisskyline,NFLX,Netflix,http://youtu.be/MLmX7WMEBVM?a,False


In [45]:
#check size
Netflix_text_data.shape

(101, 8)

In [46]:
#check data type of columns
Netflix_text_data.dtypes

id                int64
text             object
timestamp        object
source           object
symbols          object
company_names    object
url              object
verified           bool
dtype: object

In [49]:
#check timestamp range
print(min(Netflix_text_data['timestamp']))
print(max(Netflix_text_data['timestamp']))

Wed Jul 18 21:07:19 +0000 2018
Wed Jul 18 23:45:04 +0000 2018


* The texts spanned a single across a period of two hours

In [58]:
#check for unique ids
len(Netflix_text_data['id'].unique())

101

* There 101 unique ids. That means each text can identified as coming from a different source.
* Let's corroborate this with checking by source

In [62]:
#check how many texts were collected from each identified source in the data set
Netflix_text_data.groupby(['source'])['text'].count()

source
10minutetrading    1
143stock           1
28delayslater      1
ABraveBull         1
AlertsChina        1
                  ..
tex_films          1
the_chart_life     1
tradnews_          1
ttttt0mmmmm        1
watchfulmkteye     1
Name: text, Length: 93, dtype: int64

In [63]:
#check if any source has more than one text
pd.DataFrame(Netflix_text_data.groupby(['source'])['text'].count()>1)

,text
source,
10minutetrading,False
143stock,False
28delayslater,False
ABraveBull,False
AlertsChina,False
...,...
tex_films,False
the_chart_life,False
tradnews_,False


In [64]:
src_frq = pd.DataFrame(Netflix_text_data.groupby(['source'])['text'].count()>1)

In [66]:
#view
src_frq.head()

,text
source,
10minutetrading,False
143stock,False
28delayslater,False
ABraveBull,False
AlertsChina,False


In [73]:
#reset index
src_frq = src_frq.reset_index()

In [74]:
#view
src_frq.head()

,source,text
0,10minutetrading,False
1,143stock,False
2,28delayslater,False
3,ABraveBull,False
4,AlertsChina,False


In [79]:
#check for true
src_frq[src_frq['text']==True]

,source,text
39,SmartTraderTips,True
79,optionalarmchat,True
87,teresaarthur66,True


* Three sources have more than one text generated from them.
* Let's check the frequency

In [86]:

src_greater_num_txt = pd.DataFrame(Netflix_text_data[(Netflix_text_data['source']=='SmartTraderTips') | (Netflix_text_data['source']=='optionalarmchat')|(Netflix_text_data['source']=='teresaarthur66')])

In [90]:
#view

src_greater_num_txt.head()


,id,text,timestamp,source,symbols,company_names,url,verified
26332,1019692061682012200,NOW OFFERING 7 Day FREE Trial to options day ...,Wed Jul 18 21:15:07 +0000 2018,optionalarmchat,NFLX,Netflix,http://ow.ly/bbVr30kY55t,False
26729,1019699616516886500,REAL TIME TRADE ALERTS via PRIVATE $TWTR FEED ...,Wed Jul 18 21:45:08 +0000 2018,optionalarmchat,NFLX,Netflix,NaN,False
26969,1019704996793905200,WTF $NFLX is not back to 423.21 yet? But $MS s...,Wed Jul 18 22:06:31 +0000 2018,SmartTraderTips,NFLX,Netflix,https://twitter.com/i/web/status/1019704996793...,False
27088,1019707164749959200,1 to 4 trade alerts with 50% to 100% profit p...,Wed Jul 18 22:15:07 +0000 2018,optionalarmchat,NFLX,Netflix,http://ow.ly/2uri30kY55u,False
27104,1019707464852394000,$NFLX $BAC $MU $FB $AAPL $MSFT $BABA $F $JD $G...,Wed Jul 18 22:16:19 +0000 2018,teresaarthur66,NFLX,Netflix,https://twitter.com/i/web/status/1019707464852...,False


In [89]:
#check size
print(src_greater_num_txt.shape)

(11, 8)


In [92]:
#check number of text from each of these three sources
src_greater_num_txt.groupby(['source']).count()['text']

source
SmartTraderTips    2
optionalarmchat    6
teresaarthur66     3
Name: text, dtype: int64

* There are 2 texts from SmartTraderTips, 6 texts from optionalarmchat, and 3 texts from teresaarthur66 within the 2hours the data was collected
